## Random.shuffle 기능을 이용하여

## training data 와 test data 분리하는 DataGeneration class 구현 

### 수치미분 함수 / 시그모이드 함수 선언

In [1]:
import numpy as np
import random
from datetime import datetime

# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

### ThoracicSurgery class 구현

In [45]:
class ThoracicSurgery:
    def __init__(self, name, input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_rate):
        self.name = name
        
        # 2층 hidden layer unit
        self.W2 = np.random.rand(input_nodes, hidden1_nodes)
        self.b2 = np.random.rand(hidden1_nodes)
        
        self.W3 = np.random.rand(hidden1_nodes, hidden2_nodes)
        self.b3 = np.random.rand(hidden2_nodes)
        
        self.W4 = np.random.rand(hidden2_nodes,output_nodes)
        self.b4 = np.random.rand(output_nodes)
        
        self.learning_rate = learning_rate
        
        print(self.name, " is created !!!")
    
    def feed_forward(self):
        delta = 1e-7
        
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        return ( np.sum( (self.target_data - y)**2 ) ) / ( len(self.input_data) )
    
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        return ( np.sum( (self.target_data - y)**2 ) ) / ( len(self.input_data) )
    
    def predict(self, input_data):
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        if y >= 0.5:
            result = 1
        else:
            result = 0
        return y, result
    
    def accuracy(self, input_data, target_data):
        matched_list = []
        not_matched_list = []
        
        index_label_prediction_list = []
        
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                index_label_prediction_list.append([index, target_data[index], logical_val])
                #temp_list.append(index)
                #temp_list.append(target_data[index])
                #temp_list.append(logical_val)
                
                #index_label_prediction_list.append(temp_list)
                
                #temp_list = []
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
    # 수치미분을 이용하여 손실함수가 최소가 될때까지 학습하닌 함수
    def train(self, input_data, target_data):
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

### DataGeneration class 구현

In [46]:
class DataGeneration:
    def __init__(self, file_path, seperation_rate, is_normalized = False):
        self.file_path = file_path
        self.seperation_rate = seperation_rate
        self.is_normalized = is_normalized
        print("DataGeneration object is Created !!!")
    #np.max() 를 이용한 데이터 정규화
    def data_normalize_using_max(self, load_data):
        
        #각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치행렬을 만들어줌
        transpose_load_data = load_data.T
        
        print("transpose_load_data.shape = ", transpose_load_data.shape)
        
        #전치행렬을 위한 리스트
        transpose_normalize_data_list = []
        
        for index in range(len(transpose_load_data)):
            
            max_value = np.max(transpose_load_data[index, :]) 
            #최대값이 1 이상이면 최대값으로 나누어서 모든 데이터가 0~1사이에 오도록 함
            if max_value > 1.0:
                
                transpose_normalize_data_list.append(transpose_load_data[index, :] / max_value)
                
                #최대값이 1보다 작으면 해당 값을 그대로 사용함.
                # 왜냐하면 1보다 작은 값이면 굳이 바꿔줄 필요가 없음
            else:
                transpose_normalize_data_list.append(transpose_load_data[index, :])
        
        # 리스트를 numpy type으로 변환
        transpose_normalize_data = np.array(transpose_normalize_data_list)
        
        print(transpose_load_data.shape)
        
        #데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
        normalize_data = transpose_normalize_data.T
        
        print(normalize_data.shape)
        
        #nomalize 파일 저장
        np.savetxt('./Normalize_Diabetes_data.csv', normalize_data, delimiter= ',')
        
        return normalize_data
    
    def generate(self):
        
        # 데이터 불러오기
        
        load_data =np.loadtxt(self.file_path, delimiter = ',', dtype = np.float32)
        
        print("[debug. before data normalization] load_data[0] = ", load_data[0])
        
        print("load_data.shape = ", load_data.shape)
        
        if(self.is_normalized == True):
            
            load_data = self.data_normalize_using_max(load_data)
            
            print("[debug. after data normalization] load_data[0] = ", load_data[0])
        
        #임시 저장 리스트
        training_data_list = []
        test_data_list = []
        
        #분리비율에 맞게 테스트 데이터로 분리
        total_data_num = len(load_data)
        test_data_num = int(len(load_data) * self.seperation_rate)
        
        print("total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)
        
        #전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num)]
        
        # random.shuffle을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list) #전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다.
        
        # test data를 위한 인덱스는 total_data_index_list 로부터 앞에서 40%의 데이터 인데스
        test_data_index_list = total_data_index_list[ 0 : test_data_num]
        
        print("length of test_data_index_list = ", len(test_data_index_list))
        
        #training data를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[test_data_num: ]
        
        print("length of training_data_index_list = ", len(training_data_index_list))
        
        # training data 구성
        for training_data_index in training_data_index_list:
            training_data_list.append(load_data[training_data_index])
        
        # test data 구성
        for test_data_index in test_data_index_list:
            test_data_list.append(load_data[test_data_index])
        
        training_data = np.array(training_data_list)
        
        test_data = np.array(test_data_list)
        
        print("training_data.shape = ", training_data.shape)
        print("test_data.shape = ", test_data.shape)
        
        np.savetxt('./random_ThoracicSurgery_training_data.csv', training_data, delimiter = ',')
        np.savetxt('./random_ThoracicSurgery_test_data.csv', test_data, delimiter = ',')
        
        return training_data, test_data

### 정규화된 DataGeneration 객체 / ThoracicSurgery 객체 생성 및 학습

In [47]:
# 정규화된 DataGeneration 객체 생성. 
seperation_rate = 0.3
data_obj = DataGeneration('./ThoracicSurgery.csv', seperation_rate, True)
(training_data, test_data) = data_obj.generate()


#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 2  # hidden 1 nodes 개수. 
h2_nodes = 2  # hidden 2 nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate. hi_node = 2, 1e-1 에서 수렴
epochs = 20   # 반복횟수. 


# ThoracicSugery 객체 생성
ThoracicSurgery_obj = ThoracicSurgery("ThoracicSurgery", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        ThoracicSurgery_obj.train(input_data, target_data)
        
    print("epochs = ", step, "loss value = ", ThoracicSurgery_obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

DataGeneration object is Created !!!
[debug. before data normalization] load_data[0] =  [293.    1.    3.8   2.8   0.    0.    0.    0.    0.    0.   12.    0.
   0.    0.    1.    0.   62.    0. ]
load_data.shape =  (470, 18)
transpose_load_data.shape =  (18, 470)
(18, 470)
(470, 18)
[debug. after data normalization] load_data[0] =  [0.62340426 0.125      0.60317457 0.03244496 0.         0.
 0.         0.         0.         0.         0.85714287 0.
 0.         0.         1.         0.         0.7126437  0.        ]
total_data_num =  470 , test_data_num =  141
length of test_data_index_list =  141
length of training_data_index_list =  329
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)
ThoracicSurgery  is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.033094567922309526
epochs =  1 loss value =  0.03154348627522155
epochs =  2 loss value =  0.029952331866230045
epochs =  3 loss value =  0.02833781514750209
epochs =  4 loss va

In [43]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list_1, false_list_1, index_label_prediction_list) = ThoracicSurgery_obj.accuracy(test_input_data, test_target_data) 

Accuracy =>  0.8865248226950354


In [44]:
print(index_label_prediction_list)

[[0, 0.0, 0], [1, 1.0, 0], [2, 1.0, 0], [3, 0.0, 0], [4, 0.0, 0], [5, 0.0, 0], [6, 0.0, 0], [7, 0.0, 0], [8, 0.0, 0], [9, 0.0, 0], [10, 1.0, 0], [11, 0.0, 0], [12, 0.0, 0], [13, 0.0, 0], [14, 0.0, 0], [15, 0.0, 0], [16, 0.0, 0], [17, 0.0, 0], [18, 0.0, 0], [19, 0.0, 0], [20, 0.0, 0], [21, 0.0, 0], [22, 0.0, 0], [23, 0.0, 0], [24, 0.0, 0], [25, 0.0, 0], [26, 0.0, 0], [27, 0.0, 0], [28, 0.0, 0], [29, 0.0, 0], [30, 1.0, 0], [31, 0.0, 0], [32, 0.0, 0], [33, 0.0, 0], [34, 0.0, 0], [35, 0.0, 0], [36, 0.0, 0], [37, 0.0, 0], [38, 0.0, 0], [39, 0.0, 0], [40, 0.0, 0], [41, 0.0, 0], [42, 0.0, 0], [43, 0.0, 0], [44, 0.0, 0], [45, 1.0, 0], [46, 0.0, 0], [47, 0.0, 0], [48, 0.0, 0], [49, 0.0, 0], [50, 0.0, 0], [51, 1.0, 0], [52, 0.0, 0], [53, 1.0, 0], [54, 0.0, 0], [55, 0.0, 0], [56, 0.0, 0], [57, 0.0, 0], [58, 0.0, 0], [59, 0.0, 0], [60, 0.0, 0], [61, 0.0, 0], [62, 0.0, 0], [63, 0.0, 0], [64, 0.0, 0], [65, 0.0, 0], [66, 0.0, 0], [67, 0.0, 0], [68, 0.0, 0], [69, 0.0, 0], [70, 0.0, 0], [71, 0.0, 0], [